# Aviation Accidents Analysis

You are part of a consulting firm that is tasked to do an analysis of commercial and passenger jet airline safety. The client (an airline/airplane insurer) is interested in knowing what types of aircraft (makes/models) exhibit low rates of total destruction and low likelihood of fatal or serious passenger injuries in the event of an accident. They are also interested in any general variables/conditions that might be at play. Your analysis will be based off of aviation accident data accumulated from the years 1948-2023. 

Our client is only interested in airplane makes/models that are professional builds and could potentially still be active. Assume a max lifetime of 40 years for a make/model retirement and make sure to filter your data accordingly (i.e. from 1983 onwards). They would also like separate recommendations for small aircraft vs. larger passenger models. **In addition, make sure that claims that you make are statistically robust and that you have enough samples when making comparisons between groups.**


In this summative assessment you will demonstrate your ability to:
- **Use Pandas to load, inspect, and clean the dataset appropriately.**
- **Transform relevant columns to create measures that address the problem at hand.**
- conduct EDA: visualization and statistical measures to systematically understand the structure of the data
- recommend a set of airplanes and makes conforming to the client's request and identify at least *two* factors contributing to airplane safety. You must provide supporting evidence (visuals, summary statistics, tables) for each claim you make.

### Make relevant library imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading and Inspection

### Load in data from the relevant directory and inspect the dataframe.
- inspect NaNs, datatypes, and summary statistics

In [2]:
# Read CSV file
data = pd.read_csv('data/AviationData.csv', encoding = 'ISO-8859-1')

# Examine the data
data.info()

'''Initial Notes:
* Event.Date and Publication.Date should be converted to a date type.
* Amateur.Built should be converted to a boolean type.
* Total.Fatal.Injuries, Total.Serious.Injuries, Total.Minor.Injuries, and Total.Uninjured should be converted to integer types.

* There are a significant number of missing values for Latitude, Longitude, Aircraft.Category, FAR.Description, Schedule, and Air.carrier.
* There are many missing values for Airport.Code, Airport.Name, Total.Fatal.Injuries, Total.Serious.Injuries, Total.Minor.Injuries, Broad.phase.of.flight, and Publication.Date.
* There are a few additional columns with missing values, but these can be addressed as needed once the data is filtered by which rows are relevant to the business problem.
'''

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50132 non-null  object 
 9   Airport.Name            52704 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87507 non-null  object 
 14  Make                    88826 non-null

/var/folders/gq/b1tckqn91cvbfg9bjycs6t4c0000gp/T/ipykernel_74235/3147097388.py:2: DtypeWarning: Columns (6,7,28) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data/AviationData.csv', encoding = 'ISO-8859-1')


'Initial Notes:\n* Event.Date and Publication.Date should be converted to a date type.\n* Amateur.Built should be converted to a boolean type.\n* Total.Fatal.Injuries, Total.Serious.Injuries, Total.Minor.Injuries, and Total.Uninjured should be converted to integer types.\n\n* There are a significant number of missing values for Latitude, Longitude, Aircraft.Category, FAR.Description, Schedule, and Air.carrier.\n* There are many missing values for Airport.Code, Airport.Name, Total.Fatal.Injuries, Total.Serious.Injuries, Total.Minor.Injuries, Broad.phase.of.flight, and Publication.Date.\n* There are a few additional columns with missing values, but these can be addressed as needed once the data is filtered by which rows are relevant to the business problem.\n'

In [3]:
data.describe()

'''Initial Notes:
* Since 1 is the first quartile, median, and third quartile for Number.of.Engines, we know that the middle half of aircraft have 1 engine, and other values are outliers.
* Since 0 is the first quartile, median, and third quartile for Total.Fatal.Injuries, Total.Series.Injuries, and Total.Minor.Injuries, we know that the middle half of aircraft have 0 injuries.
'''

'Initial Notes:\n* Since 1 is the first quartile, median, and third quartile for Number.of.Engines, we know that the middle half of aircraft have 1 engine, and other values are outliers.\n* Since 0 is the first quartile, median, and third quartile for Total.Fatal.Injuries, Total.Series.Injuries, and Total.Minor.Injuries, we know that the middle half of aircraft have 0 injuries.\n'

In [4]:
data.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


## Data Cleaning

### Filtering aircrafts and events

We want to filter the dataset to include aircraft that the client is interested in an analysis of:
- inspect relevant columns
- figure out any reasonable imputations
- filter the dataset

In [5]:
data.head()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50132 non-null  object 
 9   Airport.Name            52704 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87507 non-null  object 
 14  Make                    88826 non-null

In [6]:
# Data Filtering - Step 1

# Examine Aircraft.Category
data['Aircraft.Category'].value_counts()

# The value "UNK" can be changed to "Unknown" so these are the same category.
data['Aircraft.Category'] = data['Aircraft.Category'].replace('UNK', 'Unknown')
data['Aircraft.Category'].value_counts()

Aircraft.Category
Airplane             27617
Helicopter            3440
Glider                 508
Balloon                231
Gyrocraft              173
Weight-Shift           161
Powered Parachute       91
Ultralight              30
Unknown                 16
WSFT                     9
Powered-Lift             5
Blimp                    4
Rocket                   1
ULTR                     1
Name: count, dtype: int64

In [7]:
# Aircraft.Category has a significant number of non-null values.
# There are far more rows with Aircraft.Category == Airplane than the sum of all other categories.

# Comparing the values of Make and Model for Aircraft.Category == Nan and Aircraft.Category == Airplane show similar data.
print(data[['Aircraft.Category', 'Make', 'Model']][data['Aircraft.Category'].isna()])
print(data[['Aircraft.Category', 'Make', 'Model']][data['Aircraft.Category'] == 'Airplane'])

# Addiitionally, a quick Google search of the Makes and Models where Aircraft.Category == NaN shows that these are airplanes.

      Aircraft.Category         Make      Model
0                   NaN      Stinson      108-3
1                   NaN        Piper   PA24-180
2                   NaN       Cessna       172M
3                   NaN     Rockwell        112
4                   NaN       Cessna        501
...                 ...          ...        ...
88883               NaN  AIR TRACTOR      AT502
88884               NaN        PIPER  PA-28-151
88885               NaN     BELLANCA       7ECA
88887               NaN       CESSNA       210N
88888               NaN        PIPER  PA-24-260

[56602 rows x 3 columns]
      Aircraft.Category                        Make    Model
5              Airplane           Mcdonnell Douglas      DC9
7              Airplane                      Cessna      140
8              Airplane                      Cessna     401B
12             Airplane                    Bellanca   17-30A
13             Airplane                      Cessna    R172K
...                 ...         

In [8]:
# We can therefore reasonably assume that the missing values can be filled in as Airplane.
data['Aircraft.Category'] = data['Aircraft.Category'].fillna('Airplane')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50132 non-null  object 
 9   Airport.Name            52704 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       88889 non-null  object 
 13  Registration.Number     87507 non-null  object 
 14  Make                    88826 non-null

In [9]:
# Data Filtering - Step 2

# The client is interested in data that meets the criteria:
#   - Aircraft.Category == Airplane
#   - Amateur.Built == No
#   - Event.Date >= 1983-01-01

# Filter to aircraft that are not amateur built.
data = data[data['Amateur.Built'] == 'No']
data.info()

# This reduces the count from 88,889 entries to 80,312 entries.

<class 'pandas.core.frame.DataFrame'>
Index: 80312 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                80312 non-null  object 
 1   Investigation.Type      80312 non-null  object 
 2   Accident.Number         80312 non-null  object 
 3   Event.Date              80312 non-null  object 
 4   Location                80265 non-null  object 
 5   Country                 80092 non-null  object 
 6   Latitude                30170 non-null  object 
 7   Longitude               30164 non-null  object 
 8   Airport.Code            44965 non-null  object 
 9   Airport.Name            47325 non-null  object 
 10  Injury.Severity         79313 non-null  object 
 11  Aircraft.damage         77165 non-null  object 
 12  Aircraft.Category       80312 non-null  object 
 13  Registration.Number     79085 non-null  object 
 14  Make                    80266 non-null  obj

In [10]:
# Filter to aircraft where Aircraft.Category == Airplane
data = data[data['Aircraft.Category'] == 'Airplane']

data.info()

# This reduces the count from 80,312 entries to 76,008 entries.

<class 'pandas.core.frame.DataFrame'>
Index: 76008 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                76008 non-null  object 
 1   Investigation.Type      76008 non-null  object 
 2   Accident.Number         76008 non-null  object 
 3   Event.Date              76008 non-null  object 
 4   Location                75962 non-null  object 
 5   Country                 75793 non-null  object 
 6   Latitude                26610 non-null  object 
 7   Longitude               26605 non-null  object 
 8   Airport.Code            43344 non-null  object 
 9   Airport.Name            45641 non-null  object 
 10  Injury.Severity         75079 non-null  object 
 11  Aircraft.damage         73037 non-null  object 
 12  Aircraft.Category       76008 non-null  object 
 13  Registration.Number     74824 non-null  object 
 14  Make                    75963 non-null  obj

In [11]:
# Filter to aircraft with Event.Date >= 1983-01-01

# Convert data type to datetime
data['Event.Date'] = pd.to_datetime(data['Event.Date'])
data['Event.Date'].dtypes

# Filter to applicable dates
data = data[data['Event.Date'] >= '1983-01-01']
data.info()

# This reduces the count from 76,008 entries to 73,002 entries.

<class 'pandas.core.frame.DataFrame'>
Index: 73002 entries, 3600 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                73002 non-null  object        
 1   Investigation.Type      73002 non-null  object        
 2   Accident.Number         73002 non-null  object        
 3   Event.Date              73002 non-null  datetime64[ns]
 4   Location                72956 non-null  object        
 5   Country                 72793 non-null  object        
 6   Latitude                26607 non-null  object        
 7   Longitude               26602 non-null  object        
 8   Airport.Code            41828 non-null  object        
 9   Airport.Name            43712 non-null  object        
 10  Injury.Severity         72073 non-null  object        
 11  Aircraft.damage         70076 non-null  object        
 12  Aircraft.Category       73002 non-null  object  

### Cleaning and constructing Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

**Construct metric for fatal/serious injuries**

*Hint:* Estimate the total number of passengers on each flight. The likelihood of serious / fatal injury can be estimated as a fraction from this.

In [12]:
# Data Cleaning - Step 1

# The total number of passengers can be estimated as the sum of Total.Fatal.Injuries + Total.Serious.Injuries + Total.Minor Injuries + Total.Uninjured, since we can assume every passenger falls into one of those categories.

data['Total.Passengers'] = data['Total.Fatal.Injuries'] + data['Total.Serious.Injuries'] + data['Total.Minor.Injuries'] + data['Total.Uninjured']
data.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,Total.Passengers
3600,20001214X42040,Accident,LAX83LA093,1983-01-01,"ARROYO GRANDE, CA",United States,NaN,NaN,NaN,NaN,...,NaN,0.0,1.0,0.0,1.0,VMC,Landing,Probable Cause,NaN,2.0
3601,20001214X42095,Accident,SEA83LA036,1983-01-01,"NEWPORT, OR",United States,NaN,NaN,ONP,NEWPORT MUNICIPAL,...,NaN,0.0,0.0,1.0,3.0,VMC,Approach,Probable Cause,NaN,4.0
3602,20001214X42067,Accident,MKC83LA056,1983-01-01,"WOODBINE, IA",United States,NaN,NaN,3YR,MUNICIPAL,...,NaN,0.0,0.0,0.0,2.0,VMC,Landing,Probable Cause,NaN,2.0
3603,20001214X42063,Accident,MKC83LA050,1983-01-01,"MARYVILLE, MO",United States,NaN,NaN,78Y,RANKIN,...,NaN,0.0,0.0,0.0,1.0,VMC,Takeoff,Probable Cause,NaN,1.0
3604,20001214X42018,Accident,LAX83FUG11,1983-01-01,"UPLAND, CA",United States,NaN,NaN,CCB,CABLE,...,NaN,0.0,0.0,2.0,0.0,VMC,Approach,Probable Cause,NaN,2.0


In [13]:
# Data Cleaning - Step 2

# The likelihood of injury can then be calculated as (Total.Fatal.Injuries + Total.Serious.Injuries) / Total.Passengers
data['Injury.Likelihood'] = (data['Total.Fatal.Injuries'] + data['Total.Serious.Injuries']) / data['Total.Passengers'] 
data.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,Total.Passengers,Injury.Likelihood
3600,20001214X42040,Accident,LAX83LA093,1983-01-01,"ARROYO GRANDE, CA",United States,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,1.0,VMC,Landing,Probable Cause,NaN,2.0,0.5
3601,20001214X42095,Accident,SEA83LA036,1983-01-01,"NEWPORT, OR",United States,NaN,NaN,ONP,NEWPORT MUNICIPAL,...,0.0,0.0,1.0,3.0,VMC,Approach,Probable Cause,NaN,4.0,0.0
3602,20001214X42067,Accident,MKC83LA056,1983-01-01,"WOODBINE, IA",United States,NaN,NaN,3YR,MUNICIPAL,...,0.0,0.0,0.0,2.0,VMC,Landing,Probable Cause,NaN,2.0,0.0
3603,20001214X42063,Accident,MKC83LA050,1983-01-01,"MARYVILLE, MO",United States,NaN,NaN,78Y,RANKIN,...,0.0,0.0,0.0,1.0,VMC,Takeoff,Probable Cause,NaN,1.0,0.0
3604,20001214X42018,Accident,LAX83FUG11,1983-01-01,"UPLAND, CA",United States,NaN,NaN,CCB,CABLE,...,0.0,0.0,2.0,0.0,VMC,Approach,Probable Cause,NaN,2.0,0.0


**Aircraft.Damage**
- identify and execute any cleaning tasks
- construct a derived column tracking whether an aircraft was destroyed or not.

In [14]:
# Data Cleaning - Step 3

# Examine the data
data['Aircraft.damage'].value_counts()
data['Aircraft.damage'].isna().sum()

2926

In [15]:
# There are 2926 null values, which can be changed to Unknown to match the existing category
data['Aircraft.damage'] = data['Aircraft.damage'].fillna('Unknown')
data['Aircraft.damage'].value_counts()

Aircraft.damage
Substantial    52547
Destroyed      14958
Unknown         3023
Minor           2474
Name: count, dtype: int64

In [16]:
# Add a column to show if the aircraft was destroyed or not, assuming Destroyed is True, and Substantial or Minor is False
# Let's make the assumption that Unknown will be categorized as False

data['Aircraft.Destroyed'] = (data['Aircraft.damage'] == 'Destroyed')
data.head(10)

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,Total.Passengers,Injury.Likelihood,Aircraft.Destroyed
3600,20001214X42040,Accident,LAX83LA093,1983-01-01,"ARROYO GRANDE, CA",United States,NaN,NaN,NaN,NaN,...,1.0,0.0,1.0,VMC,Landing,Probable Cause,NaN,2.0,0.5,False
3601,20001214X42095,Accident,SEA83LA036,1983-01-01,"NEWPORT, OR",United States,NaN,NaN,ONP,NEWPORT MUNICIPAL,...,0.0,1.0,3.0,VMC,Approach,Probable Cause,NaN,4.0,0.0,False
3602,20001214X42067,Accident,MKC83LA056,1983-01-01,"WOODBINE, IA",United States,NaN,NaN,3YR,MUNICIPAL,...,0.0,0.0,2.0,VMC,Landing,Probable Cause,NaN,2.0,0.0,False
3603,20001214X42063,Accident,MKC83LA050,1983-01-01,"MARYVILLE, MO",United States,NaN,NaN,78Y,RANKIN,...,0.0,0.0,1.0,VMC,Takeoff,Probable Cause,NaN,1.0,0.0,False
3604,20001214X42018,Accident,LAX83FUG11,1983-01-01,"UPLAND, CA",United States,NaN,NaN,CCB,CABLE,...,0.0,2.0,0.0,VMC,Approach,Probable Cause,NaN,2.0,0.0,False
3605,20001214X41951,Accident,CHI83LA074,1983-01-01,"SPRINGBROOK, WI",United States,NaN,NaN,NaN,SPRINGBROOK,...,0.0,0.0,2.0,VMC,Landing,Probable Cause,NaN,2.0,0.0,False
3606,20001214X42064,Accident,MKC83LA051,1983-01-02,"INDIANOLA, IA",United States,NaN,NaN,NaN,NaN,...,1.0,0.0,1.0,VMC,Landing,Probable Cause,NaN,2.0,0.5,False
3607,20001214X42010,Accident,LAX83FA064,1983-01-02,"MONTEREY, CA",United States,NaN,NaN,NaN,NaN,...,0.0,4.0,0.0,VMC,Takeoff,Probable Cause,NaN,4.0,0.0,True
3608,20001214X41937,Accident,CHI83FA069,1983-01-02,"GENOA CITY, WI",United States,NaN,NaN,64C,VINCENT,...,0.0,0.0,0.0,VMC,Maneuvering,Probable Cause,NaN,2.0,1.0,True
3609,20001214X41919,Accident,ATL83FA081,1983-01-02,"BEAUFORT, SC",United States,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,IMC,Cruise,Probable Cause,NaN,3.0,1.0,True


### Investigate the *Make* column
- Identify cleaning tasks here
- List cleaning tasks clearly in markdown
- Execute the cleaning tasks
- For your analysis, keep Makes with a reasonable number (you can put the threshold at 50 though lower could work as well)

In [17]:
# Data Cleaning - Step 4

# Examine the data
data['Make'].value_counts()

Make
Cessna                 20818
Piper                  11260
CESSNA                  4922
Beech                   4052
PIPER                   2839
                       ...  
Remos Aircraft GmbH        1
ULTRA FLIGHT LLC           1
Polliwagen Inc.            1
SIEMER JAMES H             1
ROYSE RALPH L              1
Name: count, Length: 1875, dtype: int64

In [18]:
# In the top 5, there are 2 examples of duplicates due to title case vs upper case. For consistency, convert all Make values to title case.
data['Make'] = data['Make'].str.title()

# Strip whitespace from values.
data['Make'] = data['Make'].str.strip()

# Many of the Makes end with variations of "Inc", "Corporation", or "Industry"; or end with "Aircraft" and these can be removed. Some are run twice to account for variations in the ordering.
data['Make'] = data['Make'].str.removesuffix('.')
data['Make'] = data['Make'].str.removesuffix('Inc')
data['Make'] = data['Make'].str.removesuffix('Corporation')
data['Make'] = data['Make'].str.removesuffix('Corp')
data['Make'] = data['Make'].str.removesuffix('Co')
data['Make'] = data['Make'].str.removesuffix('Company')
data['Make'] = data['Make'].str.removesuffix('Industry')
data['Make'] = data['Make'].str.removesuffix('Industries')
data['Make'] = data['Make'].str.removesuffix('Aircraft')
data['Make'] = data['Make'].str.removesuffix(',')
data['Make'] = data['Make'].str.strip()
data['Make'] = data['Make'].str.removesuffix(',')
data['Make'] = data['Make'].str.removesuffix('Aircraft')
data['Make'] = data['Make'].str.strip()

In [19]:
# Sort alphabetically to clearly see the variations of the same name, and combine them, making some reasonable assumptions about which Make names are actually the same.
with pd.option_context('display.max_rows', None):
    print(data['Make'].value_counts().sort_index())

# Makes that appear fewer than 150 times will be filtered out in the next step, so we can skip over combining the Makes that appear less frequently than this. If we were going to be analyzing that data as well, more robust cleaning would need to be done.
makes_to_update = {
    'Aeronca-Bubeck Irving': 'Aeronca',
    'Aeronca/Bubeck': 'Aeronca',
    'Aerospatiale Alenia': 'Aerospatiale',
    'Aerospatiale/Socata': 'Aerospatiale',
    'Airbus Industrie': 'Airbus',
    'Bellanca Citabria': 'Bellanca',
    'Boeing (Stearman)': 'Boeing',
    'Boeing - Canada (De Havilland)': 'Boeing',
    'Boeing 777-306Er': 'Boeing',
    'Boeing Commercial Airplane Gro': 'Boeing',
    'Boeing Company, Long Beach Div': 'Boeing',
    'Boeing Helicopters Div': 'Boeing',
    'Boeing Of Canada/Dehav Div': 'Boeing',
    'Boeing Stearman': 'Boeing',
    'Boeing Vertol': 'Boeing',
    'Boeing-Brown': 'Boeing',
    'Boeing-Stearman': 'Boeing',
    'Bombardier Aerospace': 'Bombardier',
    'Bombardier Canadair': 'Bombardier',
    'Bombardier Learjet': 'Bombardier',
    'Cesna': 'Cessna',
    'Cessna Aircraft': 'Cessna',
    'Cessna Ector': 'Cessna',
    'Cessna Reems': 'Cessna',
    'Cessna Reims': 'Cessna',
    'Cessna Robertson': 'Cessna',
    'Cessna Skyhawk Ii': 'Cessna',
    'Cessna Soloy': 'Cessna',
    'Cessna Wren': 'Cessna',
    'Cessna/Air Repair': 'Cessna',
    'Cessna/Weaver': 'Cessna',
    'Cirrus Design': 'Cirrus',
    'Dehavilland': 'De Havilland',
    'Dehavilland Canada': 'De Havilland',
    'Diamond Aicraft': 'Diamond',
    'Diamond Aircraft Ind': 'Diamond',
    'Diamond Aircraft Ind Gmbh': 'Diamond',
    'Diamond Aircraft Industries In': 'Diamond',
    'Diamond Aicraft Industries': 'Diamond',
    'Diamond Aircraft Industry': 'Diamond',
    'Douglas/Basler': 'Douglas',
    'Embraer Executive': 'Embraer',
    'Embraer S A': 'Embraer',
    'Embraer S.A': 'Embraer',
    'Embraer Sa': 'Embraer',
    'Embraer-Empresa Brasileira De': 'Embraer',
    'Ercoupe (Eng & Research Corp.)': 'Ercoupe',
    'Fairchild Dornier': 'Fairchild',
    'Fairchild Funk': 'Fairchild',
    'Fairchild Heli-Porter': 'Fairchild',
    'Fairchild Hiller': 'Fairchild',
    'Fairchild Merlin': 'Fairchild',
    'Fairchild Swearingen': 'Fairchild',
    'Fairchild(Howard)': 'Fairchild',
    'Fairchild-Heliporter': 'Fairchild',
    'Grumman Acft Eng': 'Grumman',
    'Grumman Acft Eng Cor': 'Grumman',
    'Grumman Acft Eng Cor-Schweizer': 'Grumman',
    'Grumman Aircraft Cor-Schweizer': 'Grumman',
    'Grumman Aircraft Eng': 'Grumman',
    'Grumman American': 'Grumman',
    'Grumman American Aviation': 'Grumman',
    'Grumman American Avn': 'Grumman',
    'Grumman American Avn.': 'Grumman',
    'Grumman Schweizer': 'Grumman',
    'Grumman-Schweizer': 'Grumman',
    'Gulfstream Aerospace': 'Gulfstream',
    'Gulfstream Aerospace Lp': 'Gulfstream',
    'Gulfstream Am Corp Comm Div': 'Gulfstream',
    'Gulfstream American': 'Gulfstream',
    'Gulfstream Schweizer A/C': 'Gulfstream',
    'Gulfstream-Schweizer': 'Gulfstream',
    'Gulfstream-Schweizer A/C': 'Gulfstream',
    'Helio Aircraft Ltd': 'Helio',
    'Hiller-Osborn': 'Hiller',
    'Hiller-Soloy': 'Hiller',
    'Hughes Aero': 'Hughes',
    'Luscombe Silvaire': 'Luscombe',
    'Maule Air': 'Maule',
    'Mcdonnell-Douglas': 'Mcdonnell Douglas',
    'Mooney Airplane': 'Mooney',
    'Mooney Airplane Co': 'Mooney',
    'Mooney Airplane Company': 'Mooney',
    'Mooney International': 'Mooney',
    'North American Aviation Div': 'North American',
    'North American Rockwell': 'North American',
    'North American-Aero Classics': 'North American',
    'North American-Barene': 'North American',
    'North American-Kenney': 'North American',
    'North American-Maslon': 'North American',
    'North American-Medore': 'North American',
    'North American/Aero Classics': 'North American',
    'North American/Schwamm': 'North American',
    'North American/Victoria Mnt Lt': 'North American',
    'Piper / Laudeman': 'Piper',
    'Piper Aerostar': 'Piper',
    'Piper Cub Crafters': 'Piper',
    'Piper Pawnee': 'Piper',
    'Piper-Harris': 'Piper',
    'Piper/Cub Crafters': 'Piper',
    'Piper/Stevens': 'Piper',
    "Piper/Wally'S Flyers": 'Piper',
    'Robinson Helicopter': 'Robinson',
    'Robinson Helicopters': 'Robinson',
    'Rockwell Commander': 'Rockwell',
    'Rockwell International': 'Rockwell',
    'Schweitzer': 'Schweizer',
    'Schweizer, N36289': 'Schweizer',
    'Taylorcraft Aviation': 'Taylorcraft',
    'Waco Classic': 'Waco'    
}

data['Make'] = data['Make'].replace(makes_to_update)

Make
177Mf Llc                             1
2021Fx3 Llc                           1
3Xtrim                                1
737                                   2
777                                   1
Aaa Aircraft Leasing                  1
Aaa Aircraft Llc                      1
Ab Sportine Aviacija                  3
Abc                                   1
Ac Mfg & Dev'T                        1
Adams                                14
Adams Balloon                         1
Adams Dennis Allen                    1
Advanced Acft                         1
Advanced Aviation                     1
Advanced Technology                   1
Adventura                             1
Aerial Productions Intl.              1
Aermacchi                             2
Aero Adventure                        2
Aero At Sp Zoo                        1
Aero Bristell                         1
Aero Commander                      411
Aero Design Eleven                    1
Aero Falcon Intl.                  

In [20]:
# We can make the assumption that if there are fewer than 150 instances of a Make, there is not enough data for analysis, so these rows can be filtered out.
make_counts = data['Make'].value_counts()
make_top_categories = make_counts[make_counts > 150].index.tolist()
data = data[data['Make'].isin(make_top_categories)]

# Check the new value counts.
print(data['Make'].value_counts())

# Check the length.
print(len(data['Make'].value_counts()))
# This reduces the number of Makes to 39, a much more reasonable number of Makes to work with.

Make
Cessna               25803
Piper                14162
Beech                 5103
Boeing                2721
Bell                  1793
Grumman               1524
Mooney                1321
Bellanca               980
Air Tractor            900
Robinson               687
Hughes                 687
Aeronca                610
Mcdonnell Douglas      584
Maule                  573
Schweizer              573
De Havilland           507
Champion               504
Cirrus                 465
Airbus                 427
Stinson                421
Rockwell               412
Aero Commander         411
Luscombe               391
North American         389
Taylorcraft            377
Aerospatiale           293
Hiller                 288
Ayres                  277
Douglas                263
Embraer                255
Ercoupe                232
Aviat                  231
Gulfstream             217
Enstrom                212
Fairchild              210
Diamond                179
Bombardier             

### Inspect Model column
- Get rid of any NaNs.
- Inspect the column and counts for each model/make. Are model labels unique to each make?
- If not, create a derived column that is a unique identifier for a given plane type.

In [21]:
# Data Cleaning - Step 5

# Remove NaNs from Model column
data = data.dropna(subset = ['Model'])

# Examine the counts of Make and Model (showing more rows than the default)
with pd.option_context('display.max_rows', None):
    print(data[['Make', 'Model']].value_counts())

Make               Model               
Cessna             152                     2230
                   172                     1650
                   172N                    1094
Piper              PA-28-140                863
Cessna             172M                     759
                   150                      752
                   172P                     665
                   182                      617
                   180                      595
Piper              PA-18-150                555
                   PA-18                    552
Cessna             150M                     551
Piper              PA-28-180                547
                   PA-28-161                527
                   PA-28-181                512
Boeing             737                      487
Cessna             150L                     435
Piper              PA-38-112                425
Beech              A36                      408
Grumman            G-164A                   400


In [22]:
# That showed that each Make has many Models, but doesn't clearly show if each Model is unique to a Make. To show this, flip the order of the columns.
with pd.option_context('display.max_rows', None):
    print(data[['Model', 'Make']].value_counts())

# Scrolling through the data, there are a few Models that appear for more than one Make.
# However, these only appear with 1 or 2 value counts, indicating a potential data entry error. We can therefore assume that Models are unique to Makes.
# Therefore, a separate column combining Make and Model isn't necessary at this point, although it may become useful later.

Model                 Make             
152                   Cessna               2230
172                   Cessna               1650
172N                  Cessna               1094
PA-28-140             Piper                 863
172M                  Cessna                759
150                   Cessna                752
172P                  Cessna                665
182                   Cessna                617
180                   Cessna                595
PA-18-150             Piper                 555
PA-18                 Piper                 552
150M                  Cessna                551
PA-28-180             Piper                 547
PA-28-161             Piper                 527
PA-28-181             Piper                 512
737                   Boeing                487
150L                  Cessna                435
PA-38-112             Piper                 425
A36                   Beech                 408
G-164A                Grumman               400


### Cleaning other columns
- there are other columns containing data that might be related to the outcome of an accident. We list a few here:
- Engine.Type
- Weather.Condition
- Number.of.Engines
- Purpose.of.flight
- Broad.phase.of.flight

Inspect and identify potential cleaning tasks in each of the above columns. Execute those cleaning tasks. 

**Note**: You do not necessarily need to impute or drop NaNs here.

In [23]:
# Data Cleaning - Step 6

# Examine Engine.Type
print(data['Engine.Type'].value_counts())
data.info()

Engine.Type
Reciprocating      53127
Turbo Prop          2591
Turbo Fan           2020
Turbo Shaft         1928
Unknown              922
Turbo Jet            429
Geared Turbofan       12
UNK                    1
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 65444 entries, 3601 to 88888
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                65444 non-null  object        
 1   Investigation.Type      65444 non-null  object        
 2   Accident.Number         65444 non-null  object        
 3   Event.Date              65444 non-null  datetime64[ns]
 4   Location                65404 non-null  object        
 5   Country                 65269 non-null  object        
 6   Latitude                23072 non-null  object        
 7   Longitude               23069 non-null  object        
 8   Airport.Code            37636 non-null  object    

In [24]:
# There are NaN values and "UNK" values, which can be changed to the existing category "Unknown"
data['Engine.Type'] = data['Engine.Type'].replace({'UNK': 'Unknown'})
data['Engine.Type'] = data['Engine.Type'].fillna('Unknown')
data['Engine.Type'].value_counts()

Engine.Type
Reciprocating      53127
Unknown             5337
Turbo Prop          2591
Turbo Fan           2020
Turbo Shaft         1928
Turbo Jet            429
Geared Turbofan       12
Name: count, dtype: int64

In [25]:
# Data Cleaning - Step 7

# Examine Weather.Condition
print(data['Weather.Condition'].value_counts())
data.info()

Weather.Condition
VMC    56384
IMC     5010
UNK      673
Unk      198
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 65444 entries, 3601 to 88888
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                65444 non-null  object        
 1   Investigation.Type      65444 non-null  object        
 2   Accident.Number         65444 non-null  object        
 3   Event.Date              65444 non-null  datetime64[ns]
 4   Location                65404 non-null  object        
 5   Country                 65269 non-null  object        
 6   Latitude                23072 non-null  object        
 7   Longitude               23069 non-null  object        
 8   Airport.Code            37636 non-null  object        
 9   Airport.Name            39432 non-null  object        
 10  Injury.Severity         64613 non-null  object        
 11  Aircraft.dam

In [26]:
# There are Nan values and "Unk" values, which can be changed to the existing category "UNK"
data['Weather.Condition'] = data['Weather.Condition'].replace({'Unk': 'UNK'})
data['Weather.Condition'] = data['Weather.Condition'].fillna('UNK')
data['Weather.Condition'].value_counts()

Weather.Condition
VMC    56384
IMC     5010
UNK     4050
Name: count, dtype: int64

In [27]:
# Data Cleaning - Step 8

# Examine Number.of.Engines
data['Number.of.Engines'].value_counts()

Number.of.Engines
1.0    51717
2.0     8911
3.0      393
4.0      340
0.0      285
Name: count, dtype: int64

In [28]:
# It doesn't make sense for a plane to have 0 engines, so this is probably being used as a placeholder for unknown.
# This value can be replaced with NaN to serve as the placeholder instead.
data['Number.of.Engines'] = data['Number.of.Engines'].replace(0, np.nan)
data['Number.of.Engines'].value_counts()

Number.of.Engines
1.0    51717
2.0     8911
3.0      393
4.0      340
Name: count, dtype: int64

In [29]:
# Data Cleaning - Step 9

# Examine Purpose.of.flight
data['Purpose.of.flight'].value_counts().sort_index()

Purpose.of.flight
ASHO                             3
Aerial Application            3847
Aerial Observation             565
Air Drop                         5
Air Race show                   19
Air Race/show                   13
Banner Tow                      98
Business                      3183
Executive/corporate            349
External Load                   11
Ferry                          597
Firefighting                    18
Flight Test                    136
Glider Tow                      35
Instructional                 8795
Other Work Use                 847
PUBS                             3
Personal                     35212
Positioning                   1179
Public Aircraft                597
Public Aircraft - Federal       46
Public Aircraft - Local         17
Public Aircraft - State         30
Skydiving                      170
Unknown                       5014
Name: count, dtype: int64

In [30]:
# There are a few categories that can be combined:
#   - ASHO, Air Race show, Air Race/show
#   - Public Aircraft, Public Aircraft - Federal, Public Aircraft - Local, Public Aircraft - State, PUBS
data['Purpose.of.flight'] = data['Purpose.of.flight'].replace({
    'ASHO': 'Air Race show',
    'Air Race/show': 'Air Race show',
    'Public Aircraft - Federal': 'Public Aircraft',
    'Public Aircraft - Local': 'Public Aircraft',
    'Public Aircraft - State': 'Public Aircraft',
    'PUBS': 'Public Aircraft'
})

# NaN values can also be converted to the existing category "Unknown"
data['Purpose.of.flight'] = data['Purpose.of.flight'].fillna('Unknown')

data['Purpose.of.flight'].value_counts()

Purpose.of.flight
Personal               35212
Unknown                 9669
Instructional           8795
Aerial Application      3847
Business                3183
Positioning             1179
Other Work Use           847
Public Aircraft          693
Ferry                    597
Aerial Observation       565
Executive/corporate      349
Skydiving                170
Flight Test              136
Banner Tow                98
Air Race show             35
Glider Tow                35
Firefighting              18
External Load             11
Air Drop                   5
Name: count, dtype: int64

In [31]:
# Data Cleaning - Step 10

# Examine Broad.phase.of.flight
data['Broad.phase.of.flight'].value_counts()

Broad.phase.of.flight
Landing        12372
Takeoff         9344
Cruise          8075
Maneuvering     5874
Approach        4894
Taxi            1604
Climb           1580
Descent         1460
Go-around       1163
Standing         762
Unknown          384
Other             69
Name: count, dtype: int64

In [32]:
# Nan values can be converted to the existing category "Unknown"
data['Broad.phase.of.flight'] = data['Broad.phase.of.flight'].fillna('Unknown')
data['Broad.phase.of.flight'].value_counts()

Broad.phase.of.flight
Unknown        18247
Landing        12372
Takeoff         9344
Cruise          8075
Maneuvering     5874
Approach        4894
Taxi            1604
Climb           1580
Descent         1460
Go-around       1163
Standing         762
Other             69
Name: count, dtype: int64

In [33]:
# Data Cleaning - Step 11

# Examine Airport.Name
data['Airport.Name'].value_counts()

# There are a number of variations of "private" can be combined into one.
data['Airport.Name'] = data['Airport.Name'].replace({
    'Private': 'PRIVATE',
    'Private Airstrip': 'PRIVATE',
    'PRIVATE STRIP': 'PRIVATE',
    'PRIVATE AIRSTRIP': 'PRIVATE'
})

data['Airport.Name'].value_counts()


Airport.Name
PRIVATE                           597
NONE                              110
MERRILL FIELD                      71
MUNICIPAL                          66
VAN NUYS                           65
                                 ... 
MARIPOSA-YOSEMITE AIRPORT           1
ALAMOSA MUNICIPAL                   1
TEMPLETON MEADOWS STRIP             1
DAWSON AIRPORT                      1
WICHITA DWIGHT D EISENHOWER NT      1
Name: count, Length: 20169, dtype: int64

### Column Removal
- inspect the dataframe and drop any columns that have too many NaNs

In [34]:
# Data Cleaning - Step 12

# Examine the data
data.info()

# These columns have less than 20,000 NaNs and can be dropped:
#   - FAR.Description
#   - Air.carrier

<class 'pandas.core.frame.DataFrame'>
Index: 65444 entries, 3601 to 88888
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                65444 non-null  object        
 1   Investigation.Type      65444 non-null  object        
 2   Accident.Number         65444 non-null  object        
 3   Event.Date              65444 non-null  datetime64[ns]
 4   Location                65404 non-null  object        
 5   Country                 65269 non-null  object        
 6   Latitude                23072 non-null  object        
 7   Longitude               23069 non-null  object        
 8   Airport.Code            37636 non-null  object        
 9   Airport.Name            39432 non-null  object        
 10  Injury.Severity         64613 non-null  object        
 11  Aircraft.damage         65444 non-null  object        
 12  Aircraft.Category       65444 non-null  object  

In [35]:
# Examine the identifying data
data.head()

# These columns serve only as identifiers and will not be useful for this analysis, and can be dropped:
#   - Event.Id
#   - Accident.Number
#   - Registration.Number
#   - Publication.Date

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,Total.Passengers,Injury.Likelihood,Aircraft.Destroyed
3601,20001214X42095,Accident,SEA83LA036,1983-01-01,"NEWPORT, OR",United States,NaN,NaN,ONP,NEWPORT MUNICIPAL,...,0.0,1.0,3.0,VMC,Approach,Probable Cause,NaN,4.0,0.0,False
3602,20001214X42067,Accident,MKC83LA056,1983-01-01,"WOODBINE, IA",United States,NaN,NaN,3YR,MUNICIPAL,...,0.0,0.0,2.0,VMC,Landing,Probable Cause,NaN,2.0,0.0,False
3603,20001214X42063,Accident,MKC83LA050,1983-01-01,"MARYVILLE, MO",United States,NaN,NaN,78Y,RANKIN,...,0.0,0.0,1.0,VMC,Takeoff,Probable Cause,NaN,1.0,0.0,False
3604,20001214X42018,Accident,LAX83FUG11,1983-01-01,"UPLAND, CA",United States,NaN,NaN,CCB,CABLE,...,0.0,2.0,0.0,VMC,Approach,Probable Cause,NaN,2.0,0.0,False
3605,20001214X41951,Accident,CHI83LA074,1983-01-01,"SPRINGBROOK, WI",United States,NaN,NaN,NaN,SPRINGBROOK,...,0.0,0.0,2.0,VMC,Landing,Probable Cause,NaN,2.0,0.0,False


In [36]:
# Examine the airport data
print(data['Airport.Code'].value_counts())
print(data['Airport.Name'].value_counts())

# There are a huge number of results that mostly have small value counts.
# We can therefore assume that the value counts are small enough that there is not enough data for meaningful statistical analysis of these columns:
#   - Airport.Code
#   - Airport.Name

Airport.Code
NONE    1161
PVT      318
APA      131
ORD      127
MRI      120
        ... 
SD1        1
TSA        1
CYYC       1
686        1
RCE        1
Name: count, Length: 8832, dtype: int64
Airport.Name
PRIVATE                           597
NONE                              110
MERRILL FIELD                      71
MUNICIPAL                          66
VAN NUYS                           65
                                 ... 
MARIPOSA-YOSEMITE AIRPORT           1
ALAMOSA MUNICIPAL                   1
TEMPLETON MEADOWS STRIP             1
DAWSON AIRPORT                      1
WICHITA DWIGHT D EISENHOWER NT      1
Name: count, Length: 20169, dtype: int64


In [37]:
# Combine these three reasons into one list of columns to remove.
columns_to_remove = [
    'FAR.Description',
    'Air.carrier',
    'Event.Id',
    'Accident.Number',
    'Registration.Number',
    'Publication.Date',
    'Airport.Code',
    'Airport.Name'
]

data.drop(columns = columns_to_remove, inplace = True)

In [38]:
data.head()

,Investigation.Type,Event.Date,Location,Country,Latitude,Longitude,Injury.Severity,Aircraft.damage,Aircraft.Category,Make,...,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Total.Passengers,Injury.Likelihood,Aircraft.Destroyed
3601,Accident,1983-01-01,"NEWPORT, OR",United States,NaN,NaN,Non-Fatal,Substantial,Airplane,Cessna,...,0.0,0.0,1.0,3.0,VMC,Approach,Probable Cause,4.0,0.0,False
3602,Accident,1983-01-01,"WOODBINE, IA",United States,NaN,NaN,Non-Fatal,Substantial,Airplane,Cessna,...,0.0,0.0,0.0,2.0,VMC,Landing,Probable Cause,2.0,0.0,False
3603,Accident,1983-01-01,"MARYVILLE, MO",United States,NaN,NaN,Non-Fatal,Substantial,Airplane,Cessna,...,0.0,0.0,0.0,1.0,VMC,Takeoff,Probable Cause,1.0,0.0,False
3604,Accident,1983-01-01,"UPLAND, CA",United States,NaN,NaN,Non-Fatal,Substantial,Airplane,Piper,...,0.0,0.0,2.0,0.0,VMC,Approach,Probable Cause,2.0,0.0,False
3605,Accident,1983-01-01,"SPRINGBROOK, WI",United States,NaN,NaN,Non-Fatal,Substantial,Airplane,Cessna,...,0.0,0.0,0.0,2.0,VMC,Landing,Probable Cause,2.0,0.0,False


### Save DataFrame to csv
- its generally useful to save data to file/server after its in a sufficiently cleaned or intermediate state
- the data can then be loaded directly in another notebook for further analysis
- this helps keep your notebooks and workflow readable, clean and modularized

In [39]:
data.to_csv('data/CleanAviationData.csv')